In [ ]:
import numpy as np
import xarray as xr
import hvplot.xarray
import glob
import holoviews as hv
hv.extension('bokeh')

# environment(on conda)

```bash
conda create -y -n holo python=3.7
conda activate holo
# https://xarray.pydata.org/en/stable/getting-started-guide/installing.html#instructions
conda install -y -c conda-forge xarray dask netCDF4 bottleneck
conda install -y -c conda-forge hvplot
conda install -y -c conda-forge selenium
conda install -y -c conda-forge firefox geckodriver
conda install -y -c conda-forge jupyter
```

# make html & png

##  read

In [ ]:
fs = glob.glob('out0*.nc')
fs.sort()
dd = []
for f in fs:
    ds = xr.open_dataset(f)
    U = ds['u'].values
    V = ds['v'].values
    Vmag = np.sqrt( U**2 + V**2) + 0.00000001
    angle = (np.pi/2.) - np.arctan2(U/Vmag, V/Vmag)
    ds['Vmag'] = (['t','x','y'], Vmag)
    ds['angle'] =(['t','x','y'], angle)
    ds = ds.drop(['u','v'])
    dd.append(ds)
    
dss = xr.concat(dd, dim="t")

In [ ]:
with open('obst.dat', mode='r', encoding='utf-8') as response:
    l = next(response)
    l = next(response)

ll = l.replace('\n','').split(',')
i1,i2,j1,j2 = np.array(ll, dtype=int)

x1 = dss['x'].values[i1]
x2 = dss['x'].values[i2]
y1 = dss['y'].values[j1]
y2 = dss['y'].values[j2]

fPoly = hv.Polygons(np.array([[x1,y1],[x2,y1],[x2,y2],[x1,y2]])).options(fill_color='green')

##  make html graph

In [ ]:
fVec = dss.hvplot.vectorfield(x='x', y='y', groupby='t', angle='angle', mag='Vmag',hover=False)
fVor = dss['vortex'].hvplot(frame_height=220, frame_width=600, x='x', y='y', cmap='bwr', clim=(-10,10)) 
g = fVor * fVec * fPoly

In [ ]:
g

### Thinning out  for github

In [ ]:
dssp = dss.isel( t=range(0,int(dss.dims['t']/2),10) )

In [ ]:
fVec = dssp.hvplot.vectorfield(x='x', y='y', groupby='t', angle='angle', mag='Vmag',hover=False)
fVor = dssp['vortex'].hvplot(frame_height=220, frame_width=600, x='x', y='y', cmap='bwr', clim=(-10,10)) 
g = fVor * fVec * fPoly

###  export html

In [ ]:
d = hvplot.save(g,'out.html')
del d

###  export png

In [ ]:
%%time
for i, t in enumerate( dssp['t'].values ):
    gp = g[t].options(title=str(np.round(t,3)) + ' sec', toolbar=None)
    d = hvplot.save(gp,'png'+str(i).zfill(8) +'.png')
del d

# make gif 

In [ ]:
from PIL import Image

In [ ]:
fs = glob.glob("*.png")
imgs = [Image.open(f) for f in fs]
# imgs = imgs[0:501:2]
# appendした画像配列をGIFにする。durationで持続時間、loopでループ数を指定可能。
d= imgs[0].save('out.gif',
             save_all=True, append_images=imgs[1:], optimize=False, duration=0.5, loop=0)
del d